In [1]:
%load_ext autoreload
%autoreload

In [2]:
from gsmodutils import GSMProject
import cobra

In [3]:
project = GSMProject('.')  # load project from current directory

In [38]:
m = project.models[1]  # load E. coli GSM iJO1366 (https://www.embopress.org/doi/full/10.1038/msb.2011.65)

In [56]:
m = project.load_model('iJO1366.json')

In [50]:
sol = m.optimize()  # maximise growth on glucose
print('growth rate (glc):',sol.objective_value)
project.save_conditions(m, 'glc_growth')

growth rate (glc): 0.9823718127269798


In [57]:
m

Name,iJO1366
Memory address,0x01deffdbe278
Number of metabolites,1805
Number of reactions,2583
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [61]:
m.exchanges.get_by_id('EX_xyl__D_e').bounds = (-10,1000)  # allow xylose uptake
sol = m.optimize()
print('growth rate (glc+xyl):',sol.objective_value)
project.save_conditions(model=m, conditions_id='glc_and_xyl_growth')

growth rate (glc+xyl): 1.8015334467298485


In [60]:
?project.save_conditions

In [52]:
m.exchanges.get_by_id('EX_glc__D_e').bounds = (0,1000)  # prevent glucose uptake
sol = m.optimize()
print('growth rate (xyl):',sol.objective_value)
project.save_conditions(m, 'xyl_growth')

growth rate (xyl): 0.80609652439725


In [8]:
# Had to run the following command to get gsmodutils shell commands to work:
# set PATH=%PATH%;C:\Users\Admin\AppData\Roaming\Python\Python35\Scripts

In [9]:
# next we want to look at the pathways involved in xylose utilisation
# authors want "...a xylose catabolic pathway that directly flows into the 
# TCA cycle without interfering glycolysis and PPP..."
m_xyl = project.load_conditions('xyl_growth',m)

In [10]:
sol = m_xyl.optimize()

In [11]:
v = sol.fluxes

In [12]:
# model reactions involved in the pentose phosphate pathway
xyl_ppp_reactions = [
    'XYLI1',  # xylose isomerase
    'XYLK',  # xylose kinase
    
    #xylulose-5p and fructose-6p to ribose-5p
    'RPE',  # ribose phosphate isomerase
    'RPI',  # ribose phoshpate epimerase
    'GND',  #
    'PGL',  #
    'G6PDH2r',  #
    'PGI',  #
    
    #xylulose-5p and ribose-5p to g3p and erythrose-4p 
    'TKT1',  # transketolase 1
    'PFK_3',  # 
    'FBA3',  #
    'TPI',  #
    
    #xylulose-5p and erythrose-4p to g3p and fructose-6p
    'TKT2',  # transketolase 2
]

In [12]:
# display fluxes of the PPP reactions when xylose is the carbon/energy source
for reaction in xyl_ppp_reactions:
    print(reaction,v[reaction])

XYLI1 10.0
XYLK 10.0
RPE -1.828074279689817
RPI -4.992205592357372
GND 3.1962171787246763
PGL 3.1962171787246763
G6PDH2r 3.1962171787246763
PGI -3.1962171787246807
TKT1 4.239662090558438
PFK_3 0.0
FBA3 0.0
TPI 4.785635817711778
TKT2 3.932263629751745


In [14]:
# we want to monitor the activity of this pathway, so an 'essential pathway' test is made
# write essential pathway test, where fluxes must be within 5% of the current flux distribution

# store reaction flux ranges in dictionary
required_fluxes={}
for reaction in xyl_ppp_reactions:
    reaction_flux = v[reaction]
    reaction_flux_margin = abs(reaction_flux*0.05)
    required_fluxes[reaction] = [reaction_flux-reaction_flux_margin,reaction_flux+reaction_flux_margin]

# write test file
project.add_essential_pathway(
    'xylose_WT_PPP',
    description='WT pentose phosphate pathway flux on xylose',
    reactions=[],
    reaction_fluxes=required_fluxes,
    conditions=['xyl_growth'],
    models=['iJO1366.json']
)

now open a terminal, cd into the project directory and try running 

``` $ gsmodutils test```

In [53]:
m_xyl_glc = project.load_conditions('glc_and_xyl_growth')

Objective reaction BIOMASS_Ec_iJO1366_core_53p95M not found. Cannot set objective


In [54]:
m_xyl_glc

Name,e_coli_core
Memory address,0x01de818fdfd0
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"cytosol, extracellular space"


In [34]:
sol = m_xyl_glc.optimize()

In [35]:
v = sol.fluxes

In [36]:
# this time get ppp flux for glc+xyl solution
required_fluxes = {}
for reaction in xyl_ppp_reactions:
    if reaction in v.keys():
        reaction_flux = v[reaction]
        reaction_flux_margin = abs(reaction_flux*0.05)
        required_fluxes[reaction] = [reaction_flux-reaction_flux_margin,reaction_flux+reaction_flux_margin]
        print(reaction,reaction_flux-reaction_flux_margin,reaction_flux,reaction_flux+reaction_flux_margin)
# write another test file
# project.add_essential_pathway(
#     'glucose_xylose_WT_PPP',
#     description='WT pentose phosphate pathway flux on glucose and xylose',
#     reactions=[],
#     reaction_fluxes=required_fluxes,
#     conditions=['glc_and_xyl_growth'],
#     models=['iJO1366.json']
# )

RPE 2.544557757982145 2.678481850507521 2.812405943032897
RPI -2.3955782487704798 -2.2815030940671237 -2.1674279393637677
GND 4.711985697345913 4.959984944574645 5.2079841918033765
PGL 4.711985697345913 4.959984944574645 5.2079841918033765
G6PDH2r 4.711985697345913 4.959984944574645 5.2079841918033765
PGI 4.617818089171986 4.860861146496827 5.103904203821668
TKT1 1.4221345693984841 1.4969837572615623 1.5718329451246404
TPI 7.103512864052273 7.477381962160287 7.851251060268301
TKT2 1.1224231885836609 1.181498093245959 1.240572997908257


In [32]:
dir(project)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_conditions_file',
 '_context_file',
 '_designs_store',
 '_json_designs',
 '_load_config',
 '_load_py_design',
 '_project_path',
 '_py_compiled_designs',
 '_py_designs',
 '_py_func_mapper',
 '_write_conditions',
 'add_essential_pathway',
 'add_model',
 'conditions',
 'conditions_schema',
 'config',
 'create_project',
 'design_path',
 'designs',
 'get_conditions',
 'get_design',
 'growth_condition',
 'iter_models',
 'list_conditions',
 'list_designs',
 'list_models',
 'load_conditions',
 'load_design',
 'load_diff',
 'load_model',
 'model',
 'models',
 'project_context_lock',
 'project_path',
 'project_tester',
 'run_tests',
 'save_conditions'

In [37]:
print(m_xyl_glc)

e_coli_core
